# Light curve analysis

## This section is largely based on the astropy documentation

The Lomb-Scargle periodogram is a commonly
used statistical tool designed to detect periodic signals in unevenly spaced
observations. The [LombScargle](https://docs.astropy.org/en/stable/timeseries/lombscargle.html) class is a unified
interface to several implementations of the Lomb-Scargle periodogram, including
a fast *O[NlogN]* implementation following the algorithm presented by Press &
Rybicki.

For a detailed practical discussion of the
Lomb-Scargle periodogram, with code examples based on ``astropy``, see
*Understanding the Lomb-Scargle Periodogram*, with associated code at
https://github.com/jakevdp/PracticalLombScargle/.

### Something preliminary definitions

We import a few things here and define a function that will be used to generate sine wave models.

In [ ]:
from astropy.time import Time
from astropy.timeseries import TimeSeries
from astropy import units as u

%matplotlib widget
import matplotlib.pyplot as plt

import numpy as np

def input_signal(time, frequencies=None, amplitudes=None):
    if frequencies is None:
        frequencies = [1, np.pi/2]
    if amplitudes is None:
        amplitudes = [1, 0.1]
        
    # Make an array of zeros for each time
    y = np.zeros_like(time)
    
    # Build a model from the amplitudes and frequencies 
    for amp, frequ in zip(amplitudes, frequencies):
        y = y + amp * np.sin(2 * np.pi * time * frequ)
        
    return y
    
# np.sin(2 * np.pi * t / freq) + 0.1 * rand.standard_normal(100)

### Example -- sine wave input

To detect periodic signals in unevenly spaced observations, consider the
following data:

In [ ]:
n_samples = 100
frequency = [1]
amplitude = [1]

rand = np.random.default_rng(42)

t = 100 * rand.random(n_samples)
noise = 0.05 * rand.standard_normal()
input_sig = input_signal(t, frequencies=frequency, amplitudes=amplitude)
y = input_sig + noise

These are 100 noisy measurements taken at irregular times, with a frequency
of 1 cycle per unit time.

In [ ]:
plt.figure()
plt.plot(t, y, 'o', label='data')
even_times = np.linspace(0, 100, num=10000)
plt.plot(even_times, input_signal(even_times, frequencies=frequency, amplitudes=amplitude), 
         label='input signal')
plt.legend(loc='upper right')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Model with single frequency')
# plt.xlim(0, 10)
plt.grid()

The Lomb-Scargle periodogram, evaluated at frequencies chosen
automatically based on the input data, can be computed as follows
using the `LombScargle` class:

In [ ]:
from astropy.timeseries import LombScargle

ls = LombScargle(t, y)
frequency, power = ls.autopower(maximum_frequency=5, samples_per_peak=20)

Plotting the result with Matplotlib gives:

In [ ]:
plt.figure()
plt.plot(frequency, power)   
plt.xlabel('Frequency (1/day)')
plt.ylabel('Power')
plt.grid()

The power is normalized so that a power of 1 represents an excellent fit of a sine wave of that frequency to the data, and zero means a sine wave of that frequency does not fit the data.

We should expect a peak at a frequency of 1 cycle per day -- that was the frequency we put in initially.

The peak is not actually quite at 1.

In [ ]:
f_max = frequency[np.argmax(power)]
print(f_max)

### Check the Lomb-Scargle model

We can use the `LombScargle` object to generate the best fitting sine wave to our data at a particular frequency. 

In [ ]:
ls_model = ls.model(t, f_max)

In [ ]:
plt.figure()
plt.plot(t, y, '.', label='data')
plt.plot(even_times, ls.model(even_times, f_max), label='Best fit from LS')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend(loc='upper right')
plt.grid()

### Residual after removing the model (aka pre-whitening)

That looks reasonably good, but let's take a look at the residual, which is the difference between the LS model and the data.

In [ ]:
residual = y - ls.model(t, f_max)

In [ ]:
plt.figure()
plt.plot(t, residual, '.')
plt.xlabel('Time (days)')
plt.ylabel('Amplitude of residual')
plt.grid()

#### Find the periodogram of the residual

It looks like there is still some regular signal in the residuals, so let's find the periodogram of the residual.

In [ ]:
frequency_resid, power_resid = LombScargle(t, residual).autopower(maximum_frequency=5, samples_per_peak=20)

In [ ]:
plt.figure()
plt.plot(frequency_resid, power_resid)      
plt.grid()
plt.xlabel('Frequency (1/day)')
plt.ylabel('Power')
plt.title('Periodogram ofresidual after removing best fit sine')

The tall peak near one day is because the frequency we got from Lomb-Scargle was close, but not quite equal to, the frequency that we put into the signal.

## Example -- two sine wave input

We repeat the exercise above, but now with an input model that is a combination of two sine waves, one with frequency 1/day and one with frequency 1.7/day. The amplitude of the second frequency we set to 1/10th the amplitude of the main sine wave

In [ ]:
n_samples = 100
frequency = [1, 1.7]
amplitude = [1, 0.1]

rand = np.random.default_rng(42)

t = 100 * rand.random(n_samples)
noise = 0.05 * rand.standard_normal()
input_sig = input_signal(t, frequencies=frequency, amplitudes=amplitude)
y = input_sig + noise

In [ ]:
plt.figure()
plt.plot(t, y, 'o', label='data')
even_times = np.linspace(0, 100, num=10000)
plt.plot(even_times, input_signal(even_times, frequencies=frequency, amplitudes=amplitude), 
         label='input signal')
plt.legend(loc='upper right')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Model with two input frequencies')
# plt.xlim(0, 10)
plt.grid()

The Lomb-Scargle periodogram, evaluated at frequencies chosen
automatically based on the input data, can be computed as follows
using the `LombScargle` class:

In [ ]:
from astropy.timeseries import LombScargle

ls = LombScargle(t, y)
frequency, power = ls.autopower(maximum_frequency=5, samples_per_peak=20)

Plotting the result with Matplotlib gives:

In [ ]:
plt.figure()
plt.plot(frequency, power)   
plt.xlabel('Frequency (1/day)')
plt.ylabel('Power')
plt.title('Periodogram, signal with two frequencies')
plt.grid()

The power is normalized so that a power of 1 represents an excellent fit of a sine wave of that frequency to the data, and zero means a sine wave of that frequency does not fit the data.

We should expect a peak at a frequency of 1 cycle per day -- that was the frequency we put in initially.

The peak is not actually quite at 1.

In [ ]:
f_max = frequency[np.argmax(power)]
print(f_max)

### Check the Lomb-Scargle model

We can use the `LombScargle` object to generate the best fitting sine wave to our data at a particular frequency. 

In [ ]:
ls_model = ls.model(t, f_max)

In [ ]:
plt.figure()
plt.plot(t, y, '.')
plt.plot(even_times, ls.model(even_times, f_max), label='Best fit from LS')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend(loc='upper right')
plt.grid()

### Residual after removing the model (aka pre-whitening)

That looks reasonably good, but let's take a look at the residual, which is the difference between the LS model and the data.

In [ ]:
residual = y - ls.model(t, f_max)

In [ ]:
plt.figure()
plt.plot(t, residual, '.')
plt.xlabel('Time (days)')
plt.ylabel('Amplitude of residual')
plt.grid()

#### Find the periodogram of the residual

It looks like there is still some regular signal in the residuals, so let's find the periodogram of the residual.

In [ ]:
frequency_resid, power_resid = LombScargle(t, residual).autopower(maximum_frequency=5, samples_per_peak=20)

In [ ]:
plt.figure()
plt.plot(frequency_resid, power_resid)      
plt.grid()

In [ ]:
f_max_resid = frequency[np.argmax(power_resid)]
f_max_resid

This is very close to the second frequency we put into to the signal, 1.7/day!